# Alluvial plot

In [12]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt 
import pickle

In [13]:
#pickle.dump(dict_results, open('country_overlap_particip.p', 'wb'))
with open('country_overlap_particip.p', 'rb') as f:
    dict_results = pickle.load(f)

with open('country_coords.p', 'rb') as f:
    coords_dict = pickle.load(f)

In [14]:
t1 = dict_results[ '1986-1988'].sort_values(by='country')
t2 = dict_results[ '1989-1991'].sort_values(by='country')

coords = coords_dict['ES']
coords['year']= coords.index
coords

list(dict_results.keys())

#coords.loc[coords.pc_w_out<1/3 & coords.z_out_overl<2] = 
#coords_dict[ 'ES']['Area']=


['1986-1988',
 '1989-1991',
 '1992-1994',
 '1995-1997',
 '1998-2000',
 '2001-2003',
 '2004-2006',
 '2007-2009',
 '2010-2012',
 '2013-2015',
 '2016-2018',
 '2019-2021']

In [15]:
year_list = list(dict_results.keys())
year_list.sort()
print('Make sure year_list has the right order (ascending):\n',year_list)

pair_list= [(year_list[i],year_list[i+1]) for i in range(len(list(dict_results.keys()))-1)]
print('Pairs:\n', pair_list)


Make sure year_list has the right order (ascending):
 ['1986-1988', '1989-1991', '1992-1994', '1995-1997', '1998-2000', '2001-2003', '2004-2006', '2007-2009', '2010-2012', '2013-2015', '2016-2018', '2019-2021']
Pairs:
 [('1986-1988', '1989-1991'), ('1989-1991', '1992-1994'), ('1992-1994', '1995-1997'), ('1995-1997', '1998-2000'), ('1998-2000', '2001-2003'), ('2001-2003', '2004-2006'), ('2004-2006', '2007-2009'), ('2007-2009', '2010-2012'), ('2010-2012', '2013-2015'), ('2013-2015', '2016-2018'), ('2016-2018', '2019-2021')]


In [16]:
dict_results['1986-1988']

,country,out_overl,in_overl,out_deg,in_deg,z_out_deg,z_out_overl,pc_out,pc_w_out
0,US,78325795.0,62904760.0,14678,8334,5.913819,9.445270,0.999166,0.929294
1,FR,54971467.0,40525774.0,15002,7320,6.054154,6.533401,0.999657,0.947318
2,NL,41979477.0,37591531.0,13197,7430,5.272349,4.913531,0.999010,0.964789
3,BR,24989015.0,3995029.0,3938,1248,1.261968,2.795123,0.995972,0.897403
4,BE-LU,22297340.0,23689265.0,9293,5627,3.581397,2.459519,0.999088,0.981165
...,...,...,...,...,...,...,...,...,...
214,SI,0.0,0.0,0,0,-0.443710,-0.320562,NaN,NaN
215,FM,0.0,0.0,0,0,-0.443710,-0.320562,NaN,NaN
216,UM-71,0.0,0.0,0,0,-0.443710,-0.320562,NaN,NaN
217,UM-67,0.0,0.0,0,0,-0.443710,-0.320562,NaN,NaN


In [17]:
import matplotlib
'rgba'+str(matplotlib.colors.to_rgba('skyblue', alpha=0.4))

'rgba(0.5294117647058824, 0.807843137254902, 0.9215686274509803, 0.4)'

In [18]:
# Using Sankey diagram 
def Area_category(coords,title):
    coords['year'] = coords.index
    coords['count']= 1
    coords[title] = 'NO' # Fill NaN

    # Area assignation:
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl<2),title] ='A1'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl<2),title] ='A2'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl<2),title] = 'A3'
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl>=2),title] ='A4'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl>=2),title] ='A5'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl>=2),title] ='A6'
    coords= coords.loc[:,['country',title]]
    return coords


def Prep_data_alluvial(dict_results,pair_list, x):
    order =['A6','A3','A2','A1','NO']
    source = dict_results[pair_list[x][0]]
    target = dict_results[pair_list[x][1]]

    source_out = Area_category(source, 'Area_s')
    target_out = Area_category(target, 'Area_t')

    transfers = pd.merge(source_out, target_out, on='country') 
    #transfers= transfers.fillna('NO')
    
    colormap = {'A1':'rgba'+str(matplotlib.colors.to_rgba('skyblue', alpha=0.4)),'A2':'rgba'+str(matplotlib.colors.to_rgba('steelblue', alpha=0.4)),'A3':'rgba'+str(matplotlib.colors.to_rgba('royalblue', alpha=0.4)),'A6': 'rgba'+str(matplotlib.colors.to_rgba('mediumblue', alpha=0.4)),'NO':'rgba'+str(matplotlib.colors.to_rgba('lightgray', alpha=0.4))}
    #print(colormap)
    #color_list = transfers.
    #transfers.loc[:,'color']=color_list.map(colormap)

    transfers.loc[:,'count']=transfers.groupby(['Area_s','Area_t']).transform('count')
    transfers = transfers.drop(columns='country').drop_duplicates().sort_values('Area_s')
    #print(transfers)
    # Colormap: 
    
    #colormap = {'A1':'skyblue','A2':'steelblue','A3':'royalblue','A6':'mediumblue','NO':'lightgray'}
    #colormap = {'A1':'rgb(68, 1, 84,0.4)','A2':'rgb(49, 104, 142,0.4)','A3':'rgb(53, 183, 121,0.4)','A6':'rgb(253, 231, 37,0.4)','NO':'rgb(211, 211, 211,0.1)'}

    transfers.loc[:,'color']=transfers.Area_s.map(colormap)
    transfers.loc[transfers.Area_s == transfers.Area_t,'color'] = 'rgba'+str(matplotlib.colors.to_rgba('lightgray', alpha=0.4))#'lightgray'

    # Replace text
    transfers['Area_s']=transfers.Area_s.astype('str') + ' ' + pair_list[x][0]
    transfers['Area_t']=transfers.Area_t.astype('str') + ' ' + pair_list[x][1]
    print(transfers)
    #transfers['Area_s']=transfers.Area_s + ' ' + str(x)
    #transfers['Area_t']=transfers.Area_t + ' ' + str(x)

    return transfers

#result_df = Prep_data_alluvial(dict_results, pair_list[0])
alluv_data = pd.concat([Prep_data_alluvial(dict_results,pair_list, x) for x in range(len(pair_list))]).reset_index(drop=True)
alluv_data

#result_df = pd.concat(coords_dict.values(), ignore_index=True)
#result_df

#sum_prov=result_df.groupby('year')['count'].transform('sum')
#print(sum_prov)

           Area_s        Area_t  count  \
11   A1 1986-1988  A1 1989-1991     16   
57   A1 1986-1988  A2 1989-1991      3   
170  A1 1986-1988  A3 1989-1991      1   
19   A2 1986-1988  A2 1989-1991     32   
61   A2 1986-1988  A3 1989-1991     10   
105  A2 1986-1988  A1 1989-1991      3   
7    A3 1986-1988  A3 1989-1991     92   
9    A3 1986-1988  A6 1989-1991      1   
36   A3 1986-1988  A2 1989-1991     12   
0    A6 1986-1988  A6 1989-1991      5   
3    A6 1986-1988  A3 1989-1991      2   
177  NO 1986-1988  NO 1989-1991     37   
192  NO 1986-1988  A1 1989-1991      4   
195  NO 1986-1988  A2 1989-1991      1   

                                                 color  
11   rgba(0.8274509803921568, 0.8274509803921568, 0...  
57   rgba(0.5294117647058824, 0.807843137254902, 0....  
170  rgba(0.5294117647058824, 0.807843137254902, 0....  
19   rgba(0.8274509803921568, 0.8274509803921568, 0...  
61   rgba(0.27450980392156865, 0.5098039215686274, ...  
105  rgba(0.274509803921568

,Area_s,Area_t,count,color
0,A1 1986-1988,A1 1989-1991,16,"rgba(0.8274509803921568, 0.8274509803921568, 0..."
1,A1 1986-1988,A2 1989-1991,3,"rgba(0.5294117647058824, 0.807843137254902, 0...."
2,A1 1986-1988,A3 1989-1991,1,"rgba(0.5294117647058824, 0.807843137254902, 0...."
3,A2 1986-1988,A2 1989-1991,32,"rgba(0.8274509803921568, 0.8274509803921568, 0..."
4,A2 1986-1988,A3 1989-1991,10,"rgba(0.27450980392156865, 0.5098039215686274, ..."
...,...,...,...,...
138,A3 2016-2018,A6 2019-2021,1,"rgba(0.2549019607843137, 0.4117647058823529, 0..."
139,A3 2016-2018,A3 2019-2021,151,"rgba(0.8274509803921568, 0.8274509803921568, 0..."
140,A3 2016-2018,A2 2019-2021,14,"rgba(0.2549019607843137, 0.4117647058823529, 0..."
141,A6 2016-2018,A6 2019-2021,10,"rgba(0.8274509803921568, 0.8274509803921568, 0..."


In [19]:
alluv_data
unique_source_targ = pd.unique(alluv_data[['Area_s','Area_t']].values.ravel('K'))
print(unique_source_targ)
mapping_dict = {k:v for v, k in enumerate(unique_source_targ)}
mapping_dict

alluv_data['Area_s']= alluv_data['Area_s'].map(mapping_dict)
alluv_data['Area_t']= alluv_data['Area_t'].map(mapping_dict)
print(alluv_data)

#alluv_data['Area_s']= pd.Categorical(alluv_data['Area_s'], unique_source_targ)
#alluv_data['Area_t']= pd.Categorical(alluv_data['Area_t'], unique_source_targ)


links_dict =alluv_data.to_dict(orient='list')
links_dict

['A1 1986-1988' 'A2 1986-1988' 'A3 1986-1988' 'A6 1986-1988'
 'NO 1986-1988' 'A1 1989-1991' 'A2 1989-1991' 'A3 1989-1991'
 'A6 1989-1991' 'NO 1989-1991' 'A1 1992-1994' 'A2 1992-1994'
 'A3 1992-1994' 'A6 1992-1994' 'NO 1992-1994' 'A1 1995-1997'
 'A2 1995-1997' 'A3 1995-1997' 'A6 1995-1997' 'NO 1995-1997'
 'A1 1998-2000' 'A2 1998-2000' 'A3 1998-2000' 'A6 1998-2000'
 'NO 1998-2000' 'A1 2001-2003' 'A2 2001-2003' 'A3 2001-2003'
 'A6 2001-2003' 'NO 2001-2003' 'A1 2004-2006' 'A2 2004-2006'
 'A3 2004-2006' 'A6 2004-2006' 'NO 2004-2006' 'A1 2007-2009'
 'A2 2007-2009' 'A3 2007-2009' 'A6 2007-2009' 'NO 2007-2009'
 'A1 2010-2012' 'A2 2010-2012' 'A3 2010-2012' 'A6 2010-2012'
 'NO 2010-2012' 'A1 2013-2015' 'A2 2013-2015' 'A3 2013-2015'
 'A6 2013-2015' 'NO 2013-2015' 'A1 2016-2018' 'A2 2016-2018'
 'A3 2016-2018' 'A6 2016-2018' 'NO 2016-2018' 'A1 2019-2021'
 'A2 2019-2021' 'A3 2019-2021' 'A6 2019-2021' 'NO 2019-2021']
     Area_s  Area_t  count                                              color
0     

{'Area_s': [0,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  6,
  6,
  6,
  7,
  7,
  7,
  7,
  8,
  9,
  9,
  9,
  9,
  10,
  10,
  10,
  11,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  13,
  14,
  15,
  15,
  15,
  15,
  16,
  16,
  16,
  16,
  17,
  17,
  18,
  18,
  19,
  19,
  19,
  20,
  20,
  20,
  20,
  21,
  21,
  21,
  21,
  22,
  22,
  22,
  22,
  23,
  23,
  24,
  24,
  25,
  25,
  25,
  25,
  26,
  26,
  26,
  27,
  27,
  27,
  28,
  29,
  29,
  29,
  30,
  30,
  30,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  33,
  34,
  34,
  35,
  35,
  35,
  36,
  36,
  36,
  37,
  37,
  37,
  38,
  39,
  39,
  40,
  40,
  40,
  41,
  41,
  41,
  42,
  42,
  42,
  43,
  43,
  44,
  44,
  45,
  45,
  45,
  46,
  46,
  46,
  47,
  47,
  48,
  49,
  50,
  50,
  51,
  51,
  52,
  52,
  52,
  53,
  54],
 'Area_t': [5,
  6,
  7,
  6,
  7,
  5,
  7,
  8,
  6,
  8,
  7,
  9,
  5,
  6,
  10,
  11,
  12,
  11,
  10,
  12,
  13,
  14,
  11,
  13,
  12,
  14,
  10,
 

In [20]:
list_node = [value[0:2] for value in unique_source_targ]
print(list_node)

colormap = {'A1':'skyblue','A2':'steelblue','A3':'royalblue','A6':'mediumblue','NO':'lightgray'}
#colormap = {'A1':'#440154FF','A2':'#414487FF','A3':'22A884FF','A6':'#FDE725FF','NO':'lightgray'}
#colormap = {'A1':'rgb(68, 1, 84,1)','A2':'rgb(49, 104, 142,1)','A3':'rgb(53, 183, 121,1)','A6':'rgb(253, 231, 37,1)','NO':'lightgray'}

color_list=list(pd.Series(list_node).map(colormap))
print(color_list)

['A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO', 'A1', 'A2', 'A3', 'A6', 'NO']
['skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray', 'skyblue', 'steelblue', 'royalblue', 'mediumblue', 'lightgray'

In [23]:
links_dict

{'Area_s': [0,
  0,
  0,
  1,
  1,
  1,
  2,
  2,
  2,
  3,
  3,
  4,
  4,
  4,
  5,
  5,
  6,
  6,
  6,
  7,
  7,
  7,
  7,
  8,
  9,
  9,
  9,
  9,
  10,
  10,
  10,
  11,
  11,
  11,
  11,
  12,
  12,
  12,
  12,
  13,
  14,
  15,
  15,
  15,
  15,
  16,
  16,
  16,
  16,
  17,
  17,
  18,
  18,
  19,
  19,
  19,
  20,
  20,
  20,
  20,
  21,
  21,
  21,
  21,
  22,
  22,
  22,
  22,
  23,
  23,
  24,
  24,
  25,
  25,
  25,
  25,
  26,
  26,
  26,
  27,
  27,
  27,
  28,
  29,
  29,
  29,
  30,
  30,
  30,
  31,
  31,
  32,
  32,
  32,
  32,
  32,
  33,
  34,
  34,
  35,
  35,
  35,
  36,
  36,
  36,
  37,
  37,
  37,
  38,
  39,
  39,
  40,
  40,
  40,
  41,
  41,
  41,
  42,
  42,
  42,
  43,
  43,
  44,
  44,
  45,
  45,
  45,
  46,
  46,
  46,
  47,
  47,
  48,
  49,
  50,
  50,
  51,
  51,
  52,
  52,
  52,
  53,
  54],
 'Area_t': [5,
  6,
  7,
  6,
  7,
  5,
  7,
  8,
  6,
  8,
  7,
  9,
  5,
  6,
  10,
  11,
  12,
  11,
  10,
  12,
  13,
  14,
  11,
  13,
  12,
  14,
  10,
 

In [24]:
import plotly.graph_objects as go
fig = go.Figure(data = [go.Sankey(
    arrangement='snap',

    node = dict(
        pad = 5, thickness =10,
        #x = np.linspace(0,0.8,len(year_list)),
        #y = np.linspace(0,1,6),
        line = dict(color = 'white', width=1),
        label= list_node,
        color = color_list
    ),

    link= dict(
        source = links_dict['Area_s'],
        target = links_dict['Area_t'],
        value = links_dict['count'],
        color= links_dict['color']
    )
    #ids = np.array([])
)
])

for x_coordinate, column_name in enumerate(year_list):
    fig.add_annotation(x=x_coordinate,y=1.08,xref="x",yref="paper",text=column_name,showarrow=False, align="center")

fig.update_layout(
  title_text="Alluvial plot", 
  xaxis={
  'showgrid': False, # thin lines in the background
  'zeroline': False, # thick line at x=0
  'visible': False,  # numbers below
  },
  yaxis={
  'showgrid': False, # thin lines in the background
  'zeroline': False, # thick line at x=0
  'visible': False,  # numbers below
  }, plot_bgcolor='rgba(0,0,0,0)', font_size=10)

fig.show()


# OLD CODE

In [ ]:
def Area_category(coords,title):
    coords['year'] = coords.index
    coords['count']=1

    # Area assignation:
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl<2),title] ='A1'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl<2),title] ='A2'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl<2),title] = 'A3'
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl>=2),title] ='A4'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl>=2),title] ='A5'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl>=2),title] ='A6'
    
    coords= coords.loc[:,['country',title]]
    return coords

source = dict_results[pair_list[0][0]]
target = dict_results[pair_list[0][1]]

source_out = Area_category(source,'Area_s')
target_out = Area_category(target,'Area_t')

transfers = pd.merge(source_out, target_out, on='country') 
transfers= transfers.fillna('NO')

transfers['count']=transfers.groupby(['Area_s','Area_t']).transform('count')

transfers = transfers.drop(columns='country').drop_duplicates().sort_values('Area_s').reset_index(drop=True)
transfers['Area_s']=transfers.Area_s+' '+ pair_list[0][0]
transfers['Area_t']=transfers.Area_t +' '+ pair_list[0][1]
transfers

In [ ]:
# Trying to use go.Parcats function
import plotly.graph_objects as go

def Area_category_p(coords,title):
    coords[title] = 'NO' # Fill NaN

    # Area assignation:
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl<2),title] ='A1'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl<2),title] ='A2'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl<2),title] = 'A3'
    coords.loc[(coords.pc_w_out<1/3) & (coords.z_out_overl>=2),title] ='A4'
    coords.loc[((coords.pc_w_out>=1/3) &(coords.pc_w_out<2/3)) & (coords.z_out_overl>=2),title] ='A5'
    coords.loc[(coords.pc_w_out>=2/3) & (coords.z_out_overl>=2),title] ='A6'
    
    coords= coords.loc[:,title]
    return coords


def Prep_data_alluvial(dict_results, year):
    data = dict_results[year]
    transfers = Area_category_p(data, str(year))
    #transfers.set_index('country', drop=True)
    
    #transfers = pd.merge(source_out, target_out, on='country') 
    #transfers= transfers.fillna('NO')
    return transfers

#

'''
transfers = Prep_data_alluvial(dict_results, year_list[0])

for x in year_list[1:-1]:
    out_transfer = Prep_data_alluvial(dict_results, x)
    transfers = pd.merge(transfers, out_transfer, on='country')
transfers
'''

transfers = pd.concat([Prep_data_alluvial(dict_results, x) for x in year_list],axis=1)
transfers


# Create dimensions
dimensions = [dict(values=transfers[label], label=label,) for label in year_list]
#dimensions = [dict(values=transfers[label], label=label) for label in year_list]

#
fig = go.Figure(data = [go.Parcats(dimensions=dimensions,
        sortpaths='backward',
        #categoryarray = ['A6','A3','A2','A1','NO'],
        #hoveron='color', hoverinfo='count',
        labelfont={'size': 15, 'family': 'Times'},
        tickfont={'size': 16, 'family': 'Times'},
        arrangement='freeform')],        

)

fig.show()